In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [2]:
subjects = ['sub-032302', 'sub-032304', 'sub-032307', 'sub-032310', 'sub-032312']
lengths = [1,2,4,6,8,10,12,14,16,18,20]
metric = ['coh','ciplv','imcoh','wpli2_debiased']
condition = "EC"
k = 3

In [13]:
#Compiling outliers by frequency, per length, per subject
#path = Path(".")
path = Path("/work/srishyla/EPOCH_tests_functional/")

metric_outliers_means = {}
metric_outliers_stds = {}

for metric in metrics:
    subject_outliers_means = {}
    subject_outliers_stds = {}

    for subject in subjects:
        length_outliers_means = {}
        length_outliers_stds = {}
        
        for length in lengths:
            xarray_EC = xr.open_dataarray(path / f'{subject}_array_{metric}_{length}_{condition}.nc')
    
            N1 = len(xarray_EC.coords["region1"])
            N2 = len(xarray_EC.coords["region2"])
    
            xr_quantiles = xarray_EC.quantile(q=[0.5, 0.75], dim=["region1", "region2", "bootstrap_samples"])
            q2 = xr_quantiles.sel(quantile=0.5).drop_vars("quantile") 
            q3 = xr_quantiles.sel(quantile=0.75).drop_vars("quantile") 
            xr_thresholds = q2 + (k*(q3-q2))
            
            xr_signif_val = (xarray_EC > xr_thresholds).sum(dim=["region1", "region2"])
            xr_signif_val = xr_signif_val/N1/N2
    
            # Using np.mean here because I want to reduce this whole xarray to a single number
            length_outliers_means[length] = float(np.mean(xr_signif_val))
            length_outliers_stds[length] = float(np.std(xr_signif_val))
            
        subject_outliers_means[subject] = length_outliers_means
        subject_outliers_stds[subject] = length_outliers_stds

    
    metric_outliers_means[metric] = subject_outliers_means
    metric_outliers_stds[metric] = subject_outliers_stds
    
print(metric_outliers_means)
print(metric_outliers_stds)